<a href="https://colab.research.google.com/github/SyedSihabUsSakib/Thesis/blob/master/BiLSTM_Two_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive (if your dataset is in Drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset.csv')  # Adjust path as needed

In [5]:
# Preprocess the data
X = df['text'].values
y = df['label'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Build the model
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/10
62/62 [==============================] - 13s 114ms/step - loss: 1.5545 - accuracy: 0.3071 - val_loss: 1.4419 - val_accuracy: 0.3955
Epoch 2/10
62/62 [==============================] - 6s 92ms/step - loss: 0.9035 - accuracy: 0.6636 - val_loss: 0.9073 - val_accuracy: 0.6636
Epoch 3/10
62/62 [==============================] - 6s 92ms/step - loss: 0.2512 - accuracy: 0.9263 - val_loss: 0.8833 - val_accuracy: 0.7227
Epoch 4/10
62/62 [==============================] - 6s 92ms/step - loss: 0.0732 - accuracy: 0.9823 - val_loss: 1.0959 - val_accuracy: 0.6955
Epoch 5/10
62/62 [==============================] - 6s 92ms/step - loss: 0.0355 - accuracy: 0.9924 - val_loss: 1.0340 - val_accuracy: 0.7455
Epoch 6/10
62/62 [==============================] - 6s 91ms/step - loss: 0.0162 - accuracy: 0.9970 - val_loss: 1.4289 - val_accuracy: 0.7091


In [6]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
class_names = le.classes_
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=class_names))

18/18 [==============================] - 2s 26ms/step
Accuracy: 0.7405

Classification Report:
               precision    recall  f1-score   support

   Cy-Flaming       0.66      0.80      0.72       127
Cy-Pull-a-Pig       0.74      0.74      0.74        91
    Cy-Racism       0.72      0.70      0.71       122
    Cy-Threat       0.78      0.72      0.75       102
 Not Bullying       0.84      0.74      0.79       109

     accuracy                           0.74       551
    macro avg       0.75      0.74      0.74       551
 weighted avg       0.75      0.74      0.74       551

